In [1]:
import os
import sys
import numpy as np
from pathlib import Path
import glob
from tqdm import tqdm
import time
from typing import List, Tuple

# Configuration
GAVD_SEQUENCES_DIR = "./GAVD-sequences"
OUTPUT_DIR = "./GAVD-hsmr-params-full"
MAX_FILES = 893  # Process files 1-893

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ Setup complete!")
print(f"📁 Input directory: {GAVD_SEQUENCES_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")
print(f"🎯 Target files: 1-{MAX_FILES}")
print(f"🎥 Total videos found: {len(list(Path(GAVD_SEQUENCES_DIR).glob('*.mp4')))}")


✅ Setup complete!
📁 Input directory: ./GAVD-sequences
📁 Output directory: ./GAVD-hsmr-params-full
🎯 Target files: 1-893
🎥 Total videos found: 1801


In [2]:
def get_sorted_video_files(video_dir: str, max_files: int = None) -> List[Path]:
    """
    Get all video files from directory and sort them alphabetically.
    
    Args:
        video_dir (str): Directory containing video files
        max_files (int): Maximum number of files to return
        
    Returns:
        List[Path]: Sorted list of video file paths
    """
    video_files = list(Path(video_dir).glob('*.mp4'))
    
    # Sort alphabetically by filename
    video_files.sort(key=lambda x: x.name)
    
    if max_files:
        video_files = video_files[:max_files]
    
    return video_files

# Get sorted list of video files
video_files = get_sorted_video_files(GAVD_SEQUENCES_DIR, MAX_FILES)

print(f"📼 Found {len(video_files)} video files")
print(f"🎯 Will process first {min(len(video_files), MAX_FILES)} files")
print(f"\n📋 First 10 files:")
for i, video in enumerate(video_files[:10], 1):
    print(f"  {i:3d}. {video.name}")
    
if len(video_files) > 10:
    print(f"  ... and {len(video_files) - 10} more files")
    
print(f"\n📋 Last 5 files (to be processed):")
for i, video in enumerate(video_files[-5:], len(video_files)-4):
    print(f"  {i:3d}. {video.name}")


📼 Found 893 video files
🎯 Will process first 893 files

📋 First 10 files:
    1. cljan9b4p00043n6ligceanyp.mp4
    2. cljanb45y00083n6lmh1qhydd.mp4
    3. cljao8kyf000d3n6l0x9kgmav.mp4
    4. cljaoak47000i3n6lsrb9rit9.mp4
    5. cljaob36l000m3n6l9xokjqww.mp4
    6. cljaobtd0000q3n6lutd71mer.mp4
    7. cljaocn4e000u3n6lgnrx469h.mp4
    8. cljaodqnu000y3n6ltaisg7tz.mp4
    9. cljaoebv700113n6lw01p9hl3.mp4
   10. cljaof3ba00153n6lcr8af6bi.mp4
  ... and 883 more files

📋 Last 5 files (to be processed):
  889. clk7wdvrc005o3n6lo9zqc9oz.mp4
  890. clk7wescc005s3n6lwpgkwfkk.mp4
  891. clk7wg3vw005x3n6lh068vbd3.mp4
  892. clk7wguxr00613n6lcojvp2w1.mp4
  893. clk7wi2pt00663n6lc98qnzvz.mp4


In [3]:
def create_empty_npy_file(video_path: Path, output_dir: Path) -> Tuple[bool, str, str]:
    """
    Create an empty .npy file for a video file with HSMR naming convention.
    
    Args:
        video_path (Path): Path to the video file
        output_dir (Path): Directory to save the .npy file
        
    Returns:
        Tuple[bool, str, str]: (success, output_path, error_message)
    """
    try:
        # Create output filename with HSMR prefix
        output_filename = f"HSMR-{video_path.stem}.npy"
        output_path = output_dir / output_filename
        
        # Check if file already exists
        if output_path.exists():
            return True, str(output_path), "File already exists"
        
        # Create empty numpy array (placeholder)
        # This creates an empty array that can be replaced later with actual HSMR data
        empty_data = np.array([], dtype=object)
        
        # Save the empty array
        np.save(output_path, empty_data)
        
        return True, str(output_path), None
        
    except Exception as e:
        return False, None, str(e)

def batch_create_empty_npy_files(video_files: List[Path], output_dir: str) -> dict:
    """
    Batch create empty .npy files for a list of video files.
    
    Args:
        video_files (List[Path]): List of video file paths
        output_dir (str): Directory to save .npy files
        
    Returns:
        dict: Processing statistics and results
    """
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print(f"🚀 Creating empty .npy files for {len(video_files)} videos")
    print(f"📁 Output directory: {output_dir}")
    print("=" * 60)
    
    results = []
    start_time = time.time()
    
    # Process with progress bar
    for i, video_path in enumerate(tqdm(video_files, desc="Creating .npy files"), 1):
        success, output_file, error = create_empty_npy_file(video_path, output_path)
        
        results.append({
            'video': video_path.name,
            'success': success,
            'output': output_file,
            'error': error,
            'index': i
        })
        
        # Print periodic updates
        if i % 100 == 0:
            successful_so_far = sum(1 for r in results if r['success'])
            print(f"✅ Progress: {i}/{len(video_files)} ({successful_so_far} successful)")
    
    end_time = time.time()
    total_time = end_time - start_time
    
    # Calculate statistics
    successful = sum(1 for r in results if r['success'])
    failed = len(results) - successful
    already_existed = sum(1 for r in results if r['success'] and r['error'] == "File already exists")
    newly_created = successful - already_existed
    
    stats = {
        'total_files': len(results),
        'successful': successful,
        'failed': failed,
        'already_existed': already_existed,
        'newly_created': newly_created,
        'total_time_seconds': total_time,
        'results': results
    }
    
    return stats

# Create empty .npy files
creation_stats = batch_create_empty_npy_files(video_files, OUTPUT_DIR)

print("\n" + "="*60)
print("📊 BATCH NPY FILE CREATION COMPLETE!")
print("="*60)
print(f"📼 Total files processed: {creation_stats['total_files']}")
print(f"✅ Successful: {creation_stats['successful']}")
print(f"❌ Failed: {creation_stats['failed']}")
print(f"📁 Already existed: {creation_stats['already_existed']}")
print(f"🆕 Newly created: {creation_stats['newly_created']}")
print(f"⏱️ Total time: {creation_stats['total_time_seconds']:.2f} seconds")

if creation_stats['failed'] > 0:
    print(f"\n❌ Failed files:")
    failed_files = [r for r in creation_stats['results'] if not r['success']]
    for failed in failed_files[:10]:  # Show first 10 failures
        print(f"  - {failed['video']}: {failed['error']}")
    if len(failed_files) > 10:
        print(f"  ... and {len(failed_files) - 10} more failures")

print(f"\n📁 Empty .npy files saved to: {OUTPUT_DIR}")
output_files = list(Path(OUTPUT_DIR).glob('*.npy'))
print(f"📄 Total .npy files in directory: {len(output_files)}")


🚀 Creating empty .npy files for 893 videos
📁 Output directory: ./GAVD-hsmr-params-full


Creating .npy files:  37%|███▋      | 333/893 [00:00<00:00, 3324.75it/s]

✅ Progress: 100/893 (100 successful)
✅ Progress: 200/893 (200 successful)
✅ Progress: 300/893 (300 successful)
✅ Progress: 400/893 (400 successful)
✅ Progress: 500/893 (500 successful)
✅ Progress: 600/893 (600 successful)


Creating .npy files:  75%|███████▍  | 666/893 [00:00<00:00, 3191.07it/s]

✅ Progress: 700/893 (700 successful)


Creating .npy files: 100%|██████████| 893/893 [00:00<00:00, 3150.20it/s]

✅ Progress: 800/893 (800 successful)

📊 BATCH NPY FILE CREATION COMPLETE!
📼 Total files processed: 893
✅ Successful: 893
❌ Failed: 0
📁 Already existed: 0
🆕 Newly created: 893
⏱️ Total time: 0.29 seconds

📁 Empty .npy files saved to: ./GAVD-hsmr-params-full
📄 Total .npy files in directory: 893


In [4]:
def verify_created_files(output_dir: str) -> dict:
    """
    Verify the created .npy files and provide statistics.
    
    Args:
        output_dir (str): Directory containing .npy files
        
    Returns:
        dict: Verification statistics
    """
    output_path = Path(output_dir)
    npy_files = list(output_path.glob('*.npy'))
    
    print(f"🔍 Verifying {len(npy_files)} .npy files...")
    
    valid_files = []
    invalid_files = []
    total_size = 0
    
    for npy_file in npy_files:
        try:
            # Try to load the file
            data = np.load(npy_file, allow_pickle=True)
            file_size = npy_file.stat().st_size
            total_size += file_size
            
            valid_files.append({
                'name': npy_file.name,
                'size_bytes': file_size,
                'data_shape': data.shape,
                'data_type': type(data).__name__
            })
            
        except Exception as e:
            invalid_files.append({
                'name': npy_file.name,
                'error': str(e)
            })
    
    stats = {
        'total_files': len(npy_files),
        'valid_files': len(valid_files),
        'invalid_files': len(invalid_files),
        'total_size_bytes': total_size,
        'total_size_mb': total_size / (1024 * 1024),
        'valid_file_details': valid_files,
        'invalid_file_details': invalid_files
    }
    
    return stats

# Verify the created files
verification_stats = verify_created_files(OUTPUT_DIR)

print("\n" + "="*60)
print("🔍 FILE VERIFICATION COMPLETE!")
print("="*60)
print(f"📄 Total files: {verification_stats['total_files']}")
print(f"✅ Valid files: {verification_stats['valid_files']}")
print(f"❌ Invalid files: {verification_stats['invalid_files']}")
print(f"💾 Total size: {verification_stats['total_size_mb']:.2f} MB")

if verification_stats['valid_files'] > 0:
    avg_size = verification_stats['total_size_bytes'] / verification_stats['valid_files']
    print(f"📊 Average file size: {avg_size:.0f} bytes")
    
    print(f"\n📋 Sample files (first 10):")
    for i, file_info in enumerate(verification_stats['valid_file_details'][:10], 1):
        print(f"  {i:2d}. {file_info['name']} ({file_info['size_bytes']} bytes, shape: {file_info['data_shape']})")
        
    if len(verification_stats['valid_file_details']) > 10:
        print(f"  ... and {len(verification_stats['valid_file_details']) - 10} more files")

if verification_stats['invalid_files'] > 0:
    print(f"\n❌ Invalid files:")
    for file_info in verification_stats['invalid_file_details']:
        print(f"  - {file_info['name']}: {file_info['error']}")

print(f"\n🎯 Success rate: {verification_stats['valid_files']/verification_stats['total_files']*100:.1f}%")
print(f"📁 Files ready for HSMR processing in: {OUTPUT_DIR}")


🔍 Verifying 893 .npy files...

🔍 FILE VERIFICATION COMPLETE!
📄 Total files: 893
✅ Valid files: 893
❌ Invalid files: 0
💾 Total size: 0.23 MB
📊 Average file size: 275 bytes

📋 Sample files (first 10):
   1. HSMR-cljan9b4p00043n6ligceanyp.npy (275 bytes, shape: (0,))
   2. HSMR-cljanb45y00083n6lmh1qhydd.npy (275 bytes, shape: (0,))
   3. HSMR-cljao8kyf000d3n6l0x9kgmav.npy (275 bytes, shape: (0,))
   4. HSMR-cljaoak47000i3n6lsrb9rit9.npy (275 bytes, shape: (0,))
   5. HSMR-cljaob36l000m3n6l9xokjqww.npy (275 bytes, shape: (0,))
   6. HSMR-cljaobtd0000q3n6lutd71mer.npy (275 bytes, shape: (0,))
   7. HSMR-cljaocn4e000u3n6lgnrx469h.npy (275 bytes, shape: (0,))
   8. HSMR-cljaodqnu000y3n6ltaisg7tz.npy (275 bytes, shape: (0,))
   9. HSMR-cljaoebv700113n6lw01p9hl3.npy (275 bytes, shape: (0,))
  10. HSMR-cljaof3ba00153n6lcr8af6bi.npy (275 bytes, shape: (0,))
  ... and 883 more files

🎯 Success rate: 100.0%
📁 Files ready for HSMR processing in: ./GAVD-hsmr-params-full


In [ ]:
print("\n" + "="*80)
print("🎉 GAVD HSMR PARAMETER FILE PREPARATION COMPLETE!")
print("="*80)

print(f"\n📊 Summary:")
print(f"  🎯 Target files (1-{MAX_FILES}): {len(video_files)} processed")
print(f"  ✅ Empty .npy files created: {verification_stats['valid_files']}")
print(f"  📁 Output directory: {OUTPUT_DIR}")
print(f"  💾 Total size: {verification_stats['total_size_mb']:.2f} MB")

print(f"\n🔄 Next Steps:")
print(f"  1. The empty .npy files are now ready in '{OUTPUT_DIR}'")
print(f"  2. You can now run HSMR processing on these files")
print(f"  3. The HSMR processing will replace the empty arrays with actual skeleton data")
print(f"  4. Each .npy file follows the naming convention: HSMR-<sequence_id>.npy")

print(f"\n💡 File Structure:")
print(f"  📁 Input videos: {GAVD_SEQUENCES_DIR}")
print(f"  📁 Output .npy files: {OUTPUT_DIR}")
print(f"  📄 Naming pattern: HSMR-<video_name_without_extension>.npy")

print(f"\n✨ Ready for HSMR processing!")
print("="*80)
